## 1. Dining philosophers problem

In [ ]:
import threading
import time
import random

# Define the number of philosophers and forks
num_philosophers = 5
num_forks = num_philosophers

# Define semaphores for the forks and the mutex
forks = [threading.Semaphore(1) for i in range(num_forks)]
mutex = threading.Semaphore(1)

# Define the philosopher thread function
def philosopher(index):
    while True:
        print(f"Philosopher {index} is thinking...")
        time.sleep(random.randint(1, 5))
        
        mutex.acquire()
        
        left_fork_index = index
        right_fork_index = (index + 1) % num_forks
        
        forks[left_fork_index].acquire()
        forks[right_fork_index].acquire()
        
        mutex.release()
        
        print(f"Philosopher {index} is eating...")
        time.sleep(random.randint(1, 5))
        
        forks[left_fork_index].release()
        forks[right_fork_index].release()

# Create a thread for each philosopher
philosopher_threads = []
for i in range(num_philosophers):
    philosopher_threads.append(threading.Thread(target=philosopher, args=(i,)))
    
# Start the philosopher threads
for thread in philosopher_threads:
    thread.start()
    
# Wait for the philosopher threads to complete
for thread in philosopher_threads:
    thread.join()

## 2. Readers/Writers problem

In [ ]:
import time
from threading import Thread, Lock, Semaphore


class SharedResource:
    MAX_READERS = 10  # put some higher value because semaphore require some max capacity

    def __init__(self):
        self.rsemaphore = Semaphore(SharedResource.MAX_READERS)
        self.wlock = Lock()


class Reader:
    def __init__(self, name, resource):
        self.name = name
        self.resource = resource

    def read(self):
        while self.resource.wlock.locked():
            continue
        self.resource.rsemaphore.acquire()
        print(f'Reader {self.name} reading data')
        time.sleep(1)
        print(f'Reader {self.name} reading done')
        self.resource.rsemaphore.release()

class Writer:
    def __init__(self, name, resource):
        self.name = name
        self.resource = resource

    def write(self):
        while self.resource.rsemaphore._value != SharedResource.MAX_READERS:  # all semaphores are released means no one is reading
            continue
        self.resource.wlock.acquire()
        print(f'Writer {self.name} writing data')
        time.sleep(2)
        print(f'Writer {self.name} writing done')
        self.resource.wlock.release()


if __name__ == '__main__':
    sr = SharedResource()
    reader1 = Reader("reader1", sr)
    reader2 = Reader("reader2", sr)
    reader3 = Reader("reader3", sr)
    writer1 = Writer("writer1", sr)
    writer2 = Writer("writer2", sr)

    Thread(target=reader1.read).start()
    Thread(target=writer1.write).start()
    time.sleep(2)
    Thread(target=reader2.read).start()
    Thread(target=writer2.write).start()
    Thread(target=reader3.read).start()

## 3. ABA problem